In [2]:
#####
# Imports and pulls that are necessary
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from shapely.geometry import Point
######

# Loading the map of Türkiye from a source that exists in GitHub
turkey_map = gpd.read_file("TUR_adm1.shp")
turkey_boundary = turkey_map.geometry.unary_union  # Get the boundary of Turkey as a single geometry

# Definition of bounds for optimization 
minx, miny, maxx, maxy = turkey_map.total_bounds

# Setting bounds for the optimization
bounds = [(minx, maxx), (miny, maxy)]

DriverError: TUR_adm1.shp: No such file or directory

In [ ]:
# Defining major cities with higher weights and their coordinate also with their coordinates and populations
#
major_cities = {
    'Istanbul': {'population': 15462452, 'coords': (28.9784, 41.0082)},
    'Izmir': {'population': 4367251, 'coords': (27.1428, 38.4237)},
    'Ankara': {'population': 5445026, 'coords': (32.8597, 39.9334)},
    'Bursa': {'population': 3056120, 'coords': (29.0611, 40.1828)},
    'Adana': {'population': 2237940, 'coords': (35.3213, 37.0000)},
    'Antalya': {'population': 2511700, 'coords': (30.7133, 36.8969)},
    'Konya': {'population': 2232863, 'coords': (32.4846, 37.8714)},
    'Kocaeli': {'population': 1970653, 'coords': (29.9336, 40.8533)},
    'Sakarya': {'population': 1028982, 'coords': (30.4033, 40.7733)},
    'Manisa': {'population': 1444770, 'coords': (27.4265, 38.6191)},
    'Aydin': {'population': 1101585, 'coords': (27.8438, 37.8450)},
    'Eskisehir': {'population': 888828, 'coords': (30.5256, 39.7767)},
    'Balikesir': {'population': 1226108, 'coords': (27.8853, 39.6484)},
    'Canakkale': {'population': 540662, 'coords': (26.4086, 40.1553)},
    'Mersin': {'population': 1872686, 'coords': (34.6442, 36.8121)},
    'Hatay': {'population': 1620466, 'coords': (36.1583, 36.1996)},
    'Isparta': {'population': 444914, 'coords': (30.5533, 37.7648)},
    'Burdur': {'population': 270796, 'coords': (30.2870, 37.7160)},
    'Diyarbakir': {'population': 1732320, 'coords': (40.2316, 37.9250)},
    'Van': {'population': 1127723, 'coords': (43.7348, 38.5012)},
    'Batman': {'population': 626319, 'coords': (41.1392, 37.8812)},
    'Gaziantep': {'population': 2097004, 'coords': (37.0662, 37.3833)},
    'Kayseri': {'population': 1424520, 'coords': (35.4853, 38.7312)},
    'Sanliurfa': {'population': 2117233, 'coords': (38.7928, 37.1671)},
    'Erzurum': {'population': 760476, 'coords': (41.2769, 39.9042)},
    'Malatya': {'population': 806156, 'coords': (38.3167, 38.3500)},
    'Samsun': {'population': 1363055, 'coords': (36.3340, 41.2867)},
    'Trabzon': {'population': 807903, 'coords': (39.7269, 41.0050)},
    'Denizli': {'population': 1049453, 'coords': (29.0875, 37.7765)},
    'Kahramanmaras': {'population': 1171432, 'coords': (36.9264, 37.5744)},
    'Sivas': {'population': 638464, 'coords': (37.0161, 39.7477)},
    'Elazig': {'population': 587960, 'coords': (39.2236, 38.6759)},
    'Ordu': {'population': 761400, 'coords': (37.7698, 40.9833)},
    'Aksaray': {'population': 429069, 'coords': (34.0297, 38.3686)},
    'Afyon': {'population': 729483, 'coords': (30.5508, 38.7625)},
    'Kirsehir': {'population': 243042, 'coords': (34.1614, 39.1458)},
    'Karaman': {'population': 254919, 'coords': (33.2351, 37.1810)},
    'Amasya': {'population': 337800, 'coords': (35.8365, 40.6536)},
    'Yozgat': {'population': 418650, 'coords': (34.8150, 39.8181)},
    'Zonguldak': {'population': 591204, 'coords': (31.7917, 41.4564)},
    'Nevsehir': {'population': 304962, 'coords': (34.7120, 38.6244)},
    'Mardin': {'population': 854716, 'coords': (40.7319, 37.3122)},
    'Usak': {'population': 370509, 'coords': (29.4058, 38.6800)},
    'Kilis': {'population': 142792, 'coords': (37.1181, 36.7161)},
    'Siirt': {'population': 331070, 'coords': (41.9447, 37.9275)},
    'Bitlis': {'population': 350021, 'coords': (42.1086, 38.3938)},
    'Rize': {'population': 344359, 'coords': (40.5152, 41.0201)},
    'Bartin': {'population': 198999, 'coords': (32.3375, 41.6358)},
    'Bingol': {'population': 281768, 'coords': (40.5125, 38.8858)},
    'Hakkari': {'population': 278218, 'coords': (43.7408, 37.5744)},
    'Karabuk': {'population': 243614, 'coords': (32.6227, 41.2048)},
    'Kars': {'population': 284923, 'coords': (43.0975, 40.6069)},
    'Kastamonu': {'population': 379405, 'coords': (33.7753, 41.3887)},
    'Kutahya': {'population': 578640, 'coords': (29.9708, 39.4214)},
    'Osmaniye': {'population': 553012, 'coords': (36.2472, 37.0742)},
    'Sinop': {'population': 218243, 'coords': (35.1553, 42.0267)},
    'Sirnak': {'population': 537762, 'coords': (42.4642, 37.4187)},
    'Tekirdag': {'population': 1086641, 'coords': (27.5167, 40.9833)},
    'Tokat': {'population': 602086, 'coords': (36.5539, 40.3167)},
    'Tunceli': {'population': 83193, 'coords': (39.5475, 39.1106)},
    'Yalova': {'population': 276050, 'coords': (29.2769, 40.6556)},
    'Duzce': {'population': 392166, 'coords': (31.1665, 40.8438)}
}

regions = {
    'Doğu Anadolu': ['Erzurum', 'Van', 'Malatya', 'Elazig', 'Kars', 'Bitlis', 'Hakkari', 'Tunceli', 'Bingol', 'Agri', 'Igdir', 'Mus', 'Ardahan'],
    'Ege': ['Izmir', 'Manisa', 'Aydin', 'Denizli', 'Mugla', 'Kütahya', 'Afyon', 'Usak'],
    'Akdeniz': ['Antalya', 'Isparta', 'Burdur', 'Adana', 'Mersin', 'Hatay', 'Osmaniye', 'Kahramanmaraş'],
    'Karadeniz': ['Samsun', 'Trabzon', 'Ordu', 'Rize', 'Giresun', 'Sinop', 'Artvin', 'Amasya', 'Tokat', 'Corum', 'Bartin', 'Kastamonu', 'Karabuk', 'Zonguldak', 'Bolu', 'Duzce'],
    'Marmara': ['Istanbul', 'Kocaeli', 'Sakarya', 'Bursa', 'Balikesir', 'Canakkale', 'Tekirdag', 'Edirne', 'Kirklareli', 'Yalova', 'Bilecik'],
    'Güneydoğu Anadolu': ['Diyarbakir', 'Gaziantep', 'Sanliurfa', 'Mardin', 'Batman', 'Siirt', 'Adiyaman', 'Kilis', 'Sirnak'],
    'İç Anadolu': ['Ankara', 'Konya', 'Eskisehir', 'Kayseri', 'Sivas', 'Nigde', 'Aksaray', 'Kirikkale', 'Kirsehir', 'Nevsehir', 'Yozgat', 'Çankırı', 'Karaman']
}

In [ ]:
# Normalizing population to use as weights between [0,1]
#
max_population = max(city['population'] for city in major_cities.values())
min_population = min(city['population'] for city in major_cities.values())
for city in major_cities.values():
    city['weight'] = (city['population'] - min_population) / (max_population - min_population)

In [ ]:
# Creating a GeoDataFrame for each region
#
region_gdfs = {}
for region, cities in regions.items():
    data = []
    for city in cities:
        if city in major_cities:
            weight = major_cities[city]['weight']
            coords = major_cities[city]['coords']
            geometry = Point(coords[0], coords[1])
        data.append({'name': city, 'geometry': geometry, 'weights': weight})
    region_gdf = gpd.GeoDataFrame(data, columns=['name', 'geometry', 'weights'])
    region_gdfs[region] = region_gdf
print(region_gdfs)

In [ ]:
# Defining the objective function for optimization
#
def objective_function(x, region_gdf):
    total_cost = 0
    total_satisfaction = 0

    for _, row in region_gdf.iterrows():

        centroid = row['geometry']
        distance = np.sqrt((centroid.x - x[0])*2 + (centroid.y - x[1])*2)
        total_cost += distance * row['weights']
        total_satisfaction += row['weights'] / (distance + 1e-5)  # To avoid division by zero

    # Combine cost and satisfaction into a single objective
    # Here we balance them with equal weights; you can adjust these as needed
    objective_value = total_cost - total_satisfaction

    return objective_value


In [ ]:
# Constraint for ensuring points if they are Türkiye
#
def constraint_function(x):
    point = Point(x[0], x[1])
    return turkey_boundary.contains(point)

In [ ]:
# Custom Nelder-Mead optimization function with constraints and etc.
#
def constrained_nelder_mead(objective, x0, constraint, args=(), options=None):
    def wrapped_objective(x):
        if constraint(x):
            return objective(x, *args)
        else:
            return np.inf  # Return a large number to penalize invalid solutions

    result = minimize(wrapped_objective, x0, method='Nelder-Mead', options=options)
    return result


In [ ]:
optimal_solutions = {}

# Running optimization for each region and storing the optimal solutions for each region
#
for region, region_gdf in region_gdfs.items():
    # Initial guess within the bounds
    x0 = np.mean(np.array(bounds), axis=1)

    # Run Nelder-Mead optimization
    result = constrained_nelder_mead(objective_function, x0, constraint_function, args=(region_gdf,), options={'maxiter': 1000, 'xatol': 1e-5, 'fatol': 1e-5})
    optimal_solutions[region] = result.x
    print(f"Optimal solution for {region}:", result.x)


In [ ]:
# Plotting the optimal distribution points on the map
#
fig, ax = plt.subplots()
turkey_map.plot(ax=ax, color='white', edgecolor='black')

# Plot each region's cities and optimal point
for region, region_gdf in region_gdfs.items():
    region_gdf.plot(ax=ax, color='pink', markersize=10)
    optimal_point = optimal_solutions[region]
    ax.scatter(optimal_point[0], optimal_point[1], color='blue', marker='x', label=f'Optimal Point for {region}')

plt.title("Optimal Distribution Points for Sanitizers in Each Region")
plt.legend()
plt.show()